sinmulation for one step 
extend for multiple steps
instead of distrbution, 
[x_1a, ..., XNa]
b also
p(x) = X
monotone function
c+, c-. u+, u-. a, b
c = cost
u = utility
a = vector
b = vector

| Concept            | Meaning                                                            |
| ------------------ | ------------------------------------------------------------------ |
| $x \in X$          | Individual score (e.g., credit score)                              |
| $g \in \{A,B\}$    | Group membership (e.g., demographic group)                         |
| $D_g(x)$           | Score distribution for group $g$                                   |
| $w_g$              | Fraction of population in group $g$                                |
| $\pi_g(x)$         | Probability of selecting an individual in group $g$ with score $x$ |
| $p(x)$             | Probability individual with score $x$ succeeds                     |
| $U_+, U_-$         | Profit on success / cost on failure                                |
| $\mathbb{E}[u(x)]$ | Expected utility for selecting score $x$                           |
| $\mathrm{OPT}(I)$  | Maximize total expected utility over all policies                  |

$\mathrm{OPT}(I) := \max_{\pi} \mathbb{E} \left[ \sum_{g \in \{A,B\}} w_g \sum_{x \in X} \pi_g(x) D_g(x) u(x) \right] 
= \max_{\pi} \sum_{g \in \{A,B\}} w_g \sum_{x \in X} \pi_g(x) D_g(x) \mathbb{E}[u(x)]$


Characterization of the Optimal Policy and the Optimal Fair Policy in the Single-Step Setting

"Proposition 1. Under Assumption 1, the optimal policy in
the single-step setting is a group-agnostic threshold policy
that selects all the individuals in category 1, i.e., individuals
with score at least −U−/(U+ −U−)."

Definition 2. For any score x ∈ X, we say x is in one of
the following categories:
- We say x is in category 1, denoted as x ∈ C1, if
E [∆(x)] ≥0 and E [u(x)] ≥0.
- We say x is in category 3, denoted as x ∈ C3, if
E [∆(x)] ≥0 and E [u(x)] < 0.
- We say x is in category 4, denoted as x ∈ C4, if
E [∆(x)] < 0 and E [u(x)] < 0. \
Note that by Assumption 2, we do not have category 2 with
E [∆(x)] < 0 and E [u(x)] ≥0.

Assumption 2. U+/U− > C+/C−. \
Assumption 2 implies that if E[u(x)] ≥0 then E[∆(x)] ≥
0. The implications of this assumption become clear when
we categorize the scores in Definition 2.

In [ ]:
import numpy as np
from scipy.stats import norm

# p : X -> [0,1]
# Sigmoid function is monotonic
def p(x):
    return 1 / (1 + np.exp(-x))

def threshold(X, x_s):
    return [x for x in X if x >= x_s]


def expected(x, plus, minus):
    return p(x) * plus + (1-p(x)) * minus

def opt_step(X, u_plus, u_minus):
    positive = []
    for x in X:
        if expected(x, u_plus, u_minus) > 0:
            positive.append(x)
    return positive

def fair_opt_step(A, B, u_plus, u_minus, alpha):
    a_threshholds = np.sort(A)
    b_threshholds = np.sort(B)
    max_util = 0
    best_a, best_b = None, None

    a_pop = len(A) / (len(A) + len(B))
    b_pop = len(B) / (len(A) + len(B))

    for i in a_threshholds:
        a = threshold(A, i)
        for j in b_threshholds:
            b = threshold(B, j)
            if (np.abs(np.mean(a) - np.mean(b))) < alpha:
                util = a_pop * expected(np.mean(a), u_plus, u_minus) + b_pop * expected(np.mean(b), u_plus, u_minus)
                max_util = max(max_util, util)
                if max_util == util:
                    best_a, best_b = a, b
            else:
                
                continue
    return (best_a, best_b, max_util)




ModuleNotFoundError: No module named 'scipy'